### **Content License Agreement**

<font color='red'><b>**WARNING**</b></font> : 본 자료는 삼성청년SW·AI아카데미의 컨텐츠 자산으로, 보안서약서에 의거하여 어떠한 사유로도 임의로 복사, 촬영, 녹음, 복제, 보관, 전송하거나 허가 받지 않은 저장매체를 이용한 보관, 제3자에게 누설, 공개 또는 사용하는 등의 무단 사용 및 불법 배포 시 법적 조치를 받을 수 있습니다.

# 과제 목표 (Objectives)

## 과제 개요

본 과제는 대규모 언어 모델(LLM)을 활용하여 **의도 분류(Intent Classification) 데이터셋을 자동으로 생성**하고, 생성된 데이터셋을 기반으로 **소형 언어 모델(SLM)을 파인튜닝(Fine-tuning)** 하는 전체 워크플로우를 구현하는 것을 목표로 합니다. 이를 통해 데이터 증강(Data Augmentation)부터 모델 경량화 및 특정 태스크 최적화까지의 과정을 직접 경험합니다.

## 과제 진행 목적 및 배경

* **LLM을 활용한 데이터셋 자동 생성(과제 추가 부분)**: `solar-pro2`와 같은 강력한 LLM을 활용하여 고품질의 의도 분류용 예시 문장을 대량으로 생성합니다. 이를 통해 데이터 구축에 드는 시간과 비용을 절감하는 능력을 기릅니다.
* **PEFT(LoRA) 기반 모델 최적화**: HuggingFace의 `SmolLM2-135M-Instruct` 모델에 LoRA(Low-Rank Adaptation) 기법을 적용하여, 적은 양의 학습 가능한 파라미터만으로 특정 분류 태스크(simple/complex)에 대한 성능을 극대화하는 방법을 학습합니다.
* **모델 경량화 및 배포 전략 이해**: 대형 모델로 생성한 데이터셋을 기반으로 소형 모델을 파인튜닝함으로써, 실제 서비스 환경에서 효율적으로 동작할 수 있는 경량 모델을 구축하고 배포하는 전략을 탐구합니다.
* **엔드투엔드(End-to-End) 파이프라인 구현**: 데이터 생성, 전처리, 모델 훈련, 평가, 그리고 배포용 모델 저장까지의 전 과정을 직접 구현하며 실무적인 MLOps 역량을 강화합니다.

## 과제 수행으로 얻어갈 수 있는 역량

* **LLM 프롬프트 엔지니어링(과제 추가 부분)**: 특정 형식(JSON)과 내용을 갖춘 데이터를 생성하기 위한 효과적인 프롬프트를 작성하는 능력.
* **합성 데이터셋 구축 및 활용**: LLM으로 생성한 합성 데이터(Synthetic Data)를 정제하고, 이를 모델 학습에 효과적으로 활용하는 전략 수립 능력.
* **PEFT(Parameter-Efficient Fine-Tuning) 기술**: LoRA를 활용하여 전체 파라미터를 재학습하지 않고도 특정 태스크에 맞게 모델을 효율적으로 튜닝하는 기술.
* **HuggingFace 및 PyTorch 활용 능력**: `transformers`, `datasets`, `peft` 등 최신 라이브러리와 PyTorch를 활용하여 모델 훈련 파이프라인을 처음부터 구현하는 능력.

## 과제 핵심 내용

1.  **데이터셋 생성(과제 추가 부분)**: `SolarChat` LLM을 사용하여 '계정 관리', '주문 결제' 등 6가지 의도(Intent)에 대한 예시 문장과 복잡도(Complexity) 레이블이 포함된 데이터셋을 생성.
2.  **모델 및 토크나이저 준비**: `HuggingFaceTB/SmolLM2-135M-Instruct` 모델과 토크나이저를 불러와 시퀀스 분류(Sequence Classification) 태스크에 맞게 설정.
3.  **LoRA 적용 및 파인튜닝**: 베이스 모델에 LoRA 설정을 적용하여 학습 가능한 파라미터 수를 최소화하고, 생성된 데이터셋으로 모델을 파인튜닝하여 'suggested_model' (small/large)을 분류하도록 학습.
4.  **모델 저장 및 배포 준비**: 학습된 LoRA 어댑터(Adapter)를 저장하고, 필요시 베이스 모델과 병합(merge)하여 추론(Inference)에 바로 사용할 수 있는 형태로 모델을 저장.

# 5-2 챕터의 흐름

5-2 챕터: 타겟 디바이스용 모델 준비 및 변환

이번 챕터에서는 모델을 타겟 디바이스에 배포할 수 있도록 준비하고, 특정 형식으로 변환하는 과정을 다룹니다.

먼저 5-2-1에서는 라우팅 제어에 사용할 플래그를 만들기 위해 데이터셋을 생성하고, 이를 활용해 소형 언어 모델(SLM)을 미세 조정(Fine-Tuning)합니다.

이어서 5-2-2에서는 준비된 PyTorch 모델을 TFLite라는 중간 형식으로 변환하는 과정을 학습합니다. 이 과정에서 정적 그래프 변환의 장점을 살펴봅니다. 더 나아가 이 챕터의 핵심은 '왜 최종 변환이 한 번 더 필요한가'를 이해하는 것입니다.

각 디바이스의 신경망 실행 엔진은 단순히 언어만 바꾼 것이 아니라, 하드웨어 구조에 맞춰 완전히 별개로 개발된 프레임워크입니다.
따라서 자연스레 모든 디바이스에서 작동하는 소스코드는 존재하지 않으며, 각 프레임워크로의 변환 자체만으로도 별도의 긴 태스크가 되는 경우가 많습니다.
TFLite로의 변환은 많은 경우 그 최종 변환의 준비가 되며, 최종 변환의 준비과정만으로 5-2-2와 같은 과정이 필요하게 됩니다.
해당 과정의 길이와 난이도 상, 이번 3주간의 정규 과정에서 다루기에는 한계가 있어 중간 단계인 TFLite에서 과정을 마무리하게 됩니다.
이러한 과정이 있다는 사실을 아는 것을 통해 TFLite 파일을 최종적으로 디바이스별 프레임워크로 다시 변환해야만 하는 이유를 명확히 이해하게 됩니다.

차후 확장적으로 목적하는 장치에 맞춰 변환 혹은 LLM 활용을 진행하시기 위해서는, 아래의 링크를 참고해 주세요.

타겟 디바이스 별 프레임워크 추가 참고 자료 :

Qualcomm - https://docs.qualcomm.com/bundle/publicresource/topics/80-63442-100/how_to_use_genie.html?product=1601111740062489 \
Android - https://ai.google.dev/edge/mediapipe/solutions/genai/llm_inference/index?hl=ko \
Raspberry Pi : Hailo(NPU) - https://hailo.ai/blog/bringing-generative-ai-to-the-edge-llm-on-hailo-10h/ \
Apple - https://machinelearning.apple.com/research/core-ml-on-device-llama \
Arduino - https://docs.m5stack.com/en/stackflow/overview \
BeagleBone - https://docs.beagleboard.org/boards/beaglebone/ai-64/index.html \
Radxa / Rockchips - https://docs.radxa.com/en/rock5/rock5b/app-development/rkllm_install




In [1]:
%pip install langchain==0.3.27
%pip install pymupdf==1.26.3
%pip install koreanize-matplotlib
%pip install datasets==4.0.0
%pip install trl
%pip install evaluate
%pip install langchain_upstage tokenizers==0.22.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.3/397.3 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.147
    Uninstalling langsmith-0.1.147:
      Successfully uninstalled langsmith-0.1.147
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.43
    Uninstalling langchain-core-0.2.43:
      Successfully uninstalled langchain-core-0.2.43
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.1.7 requires langchain-core<0.3,>=0.2.2, but you have langchain-core 0.3.79 which is incompatible.
langchain-openai 0.1.25 requires langchain-core<0.3.0,>=0.2.40, but you have langchain-core 0.3.79 which is incompatible.
  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6



필요 패키지


langchain_upstage

langchain  

pymupdf  

datasets


-----

## 과제 개요 (Assignment Overview)

### 들어가며: LLM을 활용한 데이터셋 구축과 소형 모델 최적화

최근 대규모 언어 모델(LLM)의 발전으로 고품질의 텍스트 데이터를 프로그래밍 방식으로 생성하는 것이 가능해졌습니다. 이러한 **합성 데이터셋(Synthetic Dataset)**은 특정 태스크에 맞는 학습 데이터를 구하기 어려운 경우 매우 효과적인 대안이 됩니다.

본 과제에서는 LLM을 활용하여 **의도 분류(Intent Classification)**를 위한 데이터셋을 자동으로 생성하고, 생성된 데이터로 특정 목적에 맞게 경량화된 **소형 언어 모델(SLM)**을 **PEFT(Parameter-Efficient Fine-Tuning)** 방식으로 튜닝하는 엔드투엔드(End-to-End) 파이프라인을 구현합니다.

### 과제 목차 (Table of Contents)

이 노트북은 네 가지 주요 파트로 구성되어 있습니다.

1.  **LLM을 이용한 의도 분류 데이터셋 생성**

      * `Solar` LLM에 특정 프롬프트를 입력하여, 사전 정의된 의도(Intent) 레이블과 복잡도(Complexity)를 포함하는 학습용 문장 데이터셋을 JSON 형식으로 생성합니다.

2.  **데이터셋 전처리 및 준비**

      * 생성된 데이터를 `pandas`와 `datasets` 라이브러리를 이용해 불러오고, 모델 학습에 적합한 형태로 변환합니다. 전체 데이터를 학습(Train), 검증(Validation), 테스트(Test)용으로 분할합니다.

3.  **PEFT(LoRA)를 적용한 모델 미세조정**

      * `HuggingFaceTB/SmolLM2-135M-Instruct` 모델을 기반으로, LoRA(Low-Rank Adaptation) 기법을 적용하여 적은 비용으로 효율적인 미세조정을 진행합니다. 문장의 복잡도에 따라 적절한 모델(small/large)을 추천하도록 분류 모델을 학습시킵니다.

4.  **모델 저장 및 배포 준비**

      * 학습이 완료된 모델의 LoRA 가중치(Adapter)를 저장합니다. 필요시, 베이스 모델과 가중치를 병합(merge)하여 추론 환경에 바로 배포할 수 있는 형태로 만듭니다.

# 01. 의도 분류 데이터셋 구축 (실습에 추가된 내용)

### 라이브러리 임포트 및 버전 확인
설치된 라이브러리를 임포트하고, 각 라이브러리의 버전을 출력하여 개발 환경의 재현성을 확보합니다. 이는 협업 및 디버깅 과정에서 발생할 수 있는 잠재적 호환성 문제를 예방하는 데 중요합니다.

In [2]:
import langchain
import pymupdf
import koreanize_matplotlib
import datasets
from trl import SFTConfig, SFTTrainer, setup_chat_format

def print_version(lib):
    print(lib.__name__, " == ", lib.__version__)

print_version(langchain)
print_version(pymupdf)
print_version(datasets)

langchain  ==  0.3.27
pymupdf  ==  1.26.3
datasets  ==  4.0.0


### LLM (Large Language Model) 초기화
데이터셋 생성을 위해 Upstage사의 solar-pro2 모델을 LangChain 프레임워크를 통해 초기화합니다. API 키는 보안을 위해 환경 변수 또는 Colab의 userdata 기능을 통해 안전하게 로드합니다. LLM 인스턴스는 이후 데이터 생성 프롬프트에 대한 응답을 생성하는 데 사용됩니다.

In [3]:
import os
import json
import argparse
from typing import List, Dict
from pathlib import Path
import random
import csv
import torch
from langchain_upstage import ChatUpstage


try:
    api_key = 'up_8Wp21SIv4bjbx04Cb1FEyglr1noEN'
    os.environ["SOLAR_API_KEY"] = api_key
except Exception:
    # Colab이 아니면 이미 환경변수로 설정되어 있다고 가정합니다.
    print("failed to load llm, ", Exception)

# SolarChat 초기화(사용자 제공 코드 기반)
try:
    llm= ChatUpstage(
        model="solar-pro2",
        temperature=0.2,
        api_key = api_key
    )
    HAVE_LLM = True
except Exception:
    # Solar 패키지가 설치되어 있지 않거나 Colab 환경이 아니면 LLM 자동 생성은 불가합니다.
    raise Exception("Could not load LLM, please retry.")
    llm = None
    HAVE_LLM = False


### 데이터 생성 파라미터 정의
데이터셋 생성에 필요한 핵심 파라미터를 정의합니다. MACRO_INTENTS는 생성할 데이터의 최상위 의도(Intent) 카테고리를 목록화한 것입니다. DEFAULT_ROUTER는 각 의도의 기본 복잡도를 설정하여, 추후 생성될 문장의 suggested_model 레이블을 결정하는 규칙으로 사용됩니다. SAMPLE_TEMPLATES는 LLM 사용이 불가능할 경우를 대비한 대체 데이터 생성용 템플릿입니다.

In [4]:

# --- 사용자 정의: Macro-Intent 목록 ---
MACRO_INTENTS = [
    "계정_관리",        # 계정 생성/로그인/비밀번호 등
    "주문_결제",        # 주문, 결제 수단, 결제 실패
    "배송_문의",        # 배송상태, 배송기간, 추적
    "상품_정보",        # 상품 상세, 재고, 옵션
    "기술_지원",        # 오류, 기능문의, 사용법
    "반품_환불",        # 반품절차, 환불요청
]

# 라우팅 기준: simple -> small model, complex -> large model
# 여기서는 각 Intent에 대해 기본 권장 모델을 지정할 수 있습니다.
DEFAULT_ROUTER = {
    # Intent: "simple_by_default" (True면 일반적으로 소형모델로 처리 가능)
    "계정_관리": True,
    "주문_결제": False,
    "배송_문의": True,
    "상품_정보": True,
    "기술_지원": False,
    "반품_환불": False,
}

# 샘플 문장(LLM이 없을 경우를 대비한 간단한 템플릿 예시)
SAMPLE_TEMPLATES = {
    "계정_관리": [
        "비밀번호를 잊어버렸어요. 어떻게 재설정하나요?",
        "회원 탈퇴하려면 어떻게 해야 하나요?",
        "이메일을 변경하고 싶습니다."
    ],
    "주문_결제": [
        "주문 결제 오류가 발생했어요.",
        "카드로 결제하려는데 실패합니다.",
        "쿠폰 적용은 어디서 하나요?"
    ],
    "배송_문의": [
        "내 주문 배송 상태를 알려주세요.",
        "택배사와 운송장 번호를 알고 싶어요.",
        "배송이 지연되고 있습니다."
    ],
    "상품_정보": [
        "이 제품의 사이즈는 어떻게 되나요?",
        "재고가 언제 들어오나요?",
        "상품 상세 설명을 보여주세요."
    ],
    "기술_지원": [
        "앱이 계속 충돌합니다. 로그를 어디서 확인하나요?",
        "API 호출 시 500 에러가 납니다.",
        "SDK 설치 방법을 알려주세요."
    ],
    "반품_환불": [
        "반품 신청은 어떻게 하나요?",
        "환불 처리는 얼마나 걸리나요?",
        "상품이 불량인데 교환 가능한가요?"
    ]
}

### LLM을 이용한 발화(Utterance) 생성 함수
LLM을 호출하여 특정 의도(Intent)에 해당하는 예시 문장들을 생성하는 함수 llm_generate_utterances를 정의합니다. 이 함수는 LLM에게 JSON 배열 형식으로 응답을 요청하는 프롬프트를 구성하고, 반환된 텍스트를 파싱하여 {'text': ..., 'complexity': ...} 형태의 딕셔너리 리스트로 반환합니다. LLM 호출 실패 시에는 사전에 정의된 SAMPLE_TEMPLATES를 활용하는 예외 처리 로직을 포함합니다.

In [5]:

def llm_generate_utterances(intent: str, n: int = 100) -> List[Dict]:
    """
    LLM을 사용해 intent별로 예문을 생성합니다.
    반환 형식: List of dicts: {"text":..., "complexity": "simple"/"complex"}
    프롬프트는 LLM에게 JSONL 형식으로 결과를 달라고 요청합니다.
    """
    if not HAVE_LLM:
        # LLM이 없을 때는 템플릿 기반으로 변형을 만들어 리턴합니다.
        templates = SAMPLE_TEMPLATES.get(intent, [f"{intent} 관련 문의입니다."])
        res = []
        for i in range(n):
            base = random.choice(templates)
            # 간단한 변형: 접속사나 추가정보 삽입
            if i % 5 == 0:
                text = base + " 자세히 설명해주세요."
                complexity = "complex"
            else:
                text = base
                complexity = "simple"
            res.append({"text": text, "complexity": complexity})
        return res

    # LLM이 사용 가능한 경우
    # hint. "아래의 Parsing을 통과할 수 있도록, 문제에서 주어진 "형식"에 집중하여 프롬프팅 해주세요.
    # hint. fstring을 사용하시면 갯수와 주제에 맞게 동적으로 프롬프팅하실 수 있습니다.
    # 예시는 답이 될 수 있는 하나일 뿐이고, 구동 시 확인하실 수 있다 시피 이 답 또한 생성 성공 100%를 보장하지 않습니다.
    # 만족하실만한 결과물이 나온다면 충분합니다.
    # [START CODE]
    prompt = (
        f"다음은 고객 문의의 대주제(Macro-Intent) '{intent}'에 해당하는 실제 사용자가 말할 법한 예시 문장 {n} 개를"
        "JSON 배열로 생성해 주세요. 각 항목은 'text'와 'complexity' 필드를 가지며,"
        "'complexity'는 'simple' 또는 'complex' 중 하나로 표기하세요.\n"
        "예: [{\"text\": \"...\", \"complexity\": \"simple\"}, ...]"
        "위 형식에 어긋나는 출력은 출력에서 제외합니다."
    )
    try:
        raw = llm.invoke(prompt)
        # llm.invoke의 반환 형식은 환경에 따라 다르므로 문자열로 가정
        if isinstance(raw, dict):
            # 경우에 따라 바로 파싱된 구조가 올 수 있음
            out = raw
        else:
            out = raw
        # 문자열이면 JSON 파싱 시도
        if isinstance(out.content, str):
            out_content = out.content
            out_content = out_content.strip()
            # sometimes model returns wrapped in ```json``` fences
            if out_content.startswith('```'):
                # 간단한 정제
                out_content = '\n'.join(out_content.splitlines()[1:-1])
            # parse
            parsed = json.loads(out_content)
        else:
            parsed = out
        # validation
        results = []
        for item in parsed:
            t = item.get('text') if isinstance(item, dict) else None
            c = item.get('complexity') if isinstance(item, dict) else 'simple'
            if t:
                results.append({'text': t, 'complexity': c})
        # 보장: required n items
        if len(results) < n:
            # 간단 보충 (중복 변형)
            while len(results) < n:
                pick = random.choice(results) if results else {"text": f"{intent} 관련 문의입니다.", "complexity": "simple"}
                results.append({"text": pick['text'] + "", "complexity": pick['complexity']})
        return results[:n]
    except Exception as e:
        print("LLM 생성 중 오류 발생, 템플릿 기반으로 대체합니다.", e)
        return llm_generate_utterances(intent, n)

### 추천 모델 할당 로직 정의
생성된 문장의 의도와 복잡도를 기반으로 small 또는 large 모델을 추천하는 로직을 함수로 정의합니다. 이 함수는 DEFAULT_ROUTER 규칙을 참조하여, 복잡도가 complex인 경우는 항상 large 모델을, 그렇지 않은 경우는 기본 설정에 따라 모델을 할당합니다.

In [9]:
# 문제 2: 문장의 의도(intent)와 복잡도(complexity)에 따라 추천 모델('small' 또는 'large')을 할당하는 로직을 구현합니다.
# 'complexity'가 'complex'이면 항상 'large'를 반환해야 합니다.
# 그렇지 않은 경우, DEFAULT_ROUTER 딕셔너리를 참조하여 모델을 결정합니다.

def assign_suggested_model(intent: str, complexity: str) -> str:
    """기본 라우팅 규칙에 따라 suggested_model을 결정합니다."""
    base_small = DEFAULT_ROUTER.get(intent, True)
    # 복잡한 문장은 대형 모델 권장
    if complexity == 'complex':
        return 'large'
    return 'small' if base_small else 'large'

### 전체 데이터셋 구축 파이프라인
앞서 정의한 함수들을 조합하여 전체 합성 데이터셋을 구축하는 build_dataset 함수를 정의합니다. 이 함수는 정의된 모든 MACRO_INTENTS에 대해 반복적으로 llm_generate_utterances를 호출하고, 각 결과에 assign_suggested_model을 적용하여 최종 레코드를 생성합니다. 생성된 데이터는 중복 제거 후 CSV 및 JSONL 파일 형식으로 저장됩니다. 이 과정은 데이터 생성부터 최종 파일 직렬화(Serialization)까지의 전체 파이프라인 역할을 합니다.

In [10]:
from pathlib import Path
import csv, json
from typing import List

def build_dataset(intents: List[str], per_intent: int = 100, batch_size: int = 10, out_dir: str = 'output_dataset') -> Path:
    """
    intents: 생성할 intent 목록
    per_intent: intent당 최종 생성 개수
    batch_size: 한 번에 LLM에 요청할 문장 수
    out_dir: 저장 폴더
    """
    out_path = Path(out_dir)
    out_path.mkdir(parents=True, exist_ok=True)
    records = []

    for intent in intents:
        print(f"Generating for intent: {intent} (target count={per_intent})")
        generated_count = 0
        while generated_count < per_intent:
            current_batch = min(batch_size, per_intent - generated_count)
            examples = llm_generate_utterances(intent, n=current_batch)
            for ex in examples:
                text = ex.get('text')
                complexity = ex.get('complexity', 'simple')
                suggested_model = assign_suggested_model(intent, complexity)
                records.append({
                    'intent': intent,
                    'text': text,
                    'complexity': complexity,
                    'suggested_model': suggested_model
                })
            generated_count += current_batch
            print(f"  Generated so far: {generated_count}/{per_intent}")

    # 중복 제거
    unique_texts = set()
    deduped = []
    for r in records:
        key = (r['intent'], r['text'])
        if key not in unique_texts:
            unique_texts.add(key)
            deduped.append(r)

    # 저장: CSV 및 JSONL
    csv_file = out_path / 'intent_dataset.csv'
    jsonl_file = out_path / 'intent_dataset.jsonl'
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['intent', 'text', 'complexity', 'suggested_model'])
        writer.writeheader()
        for r in deduped:
            writer.writerow(r)

    with open(jsonl_file, 'w', encoding='utf-8') as f:
        for r in deduped:
            f.write(json.dumps(r, ensure_ascii=False) + '\n')

    print(f"Saved {len(deduped)} examples -> {csv_file}, {jsonl_file}")
    return out_path


### 데이터셋 생성 실행
정의된 build_dataset 함수를 실행하여 실제 데이터셋 생성을 시작합니다. PER_INTENT 변수는 각 의도당 생성할 문장의 수를 지정합니다. 파일이 이미 존재하는 경우, 생성 과정을 건너뛰도록 조건문이 설정되어 있습니다.

In [11]:

INTENTS = MACRO_INTENTS
PER_INTENT = 100
OUT_DIR = './'
# if certain path exists
if not os.path.exists('./intent_dataset.csv'):
    build_dataset(INTENTS, per_intent=PER_INTENT, out_dir=OUT_DIR)

Generating for intent: 계정_관리 (target count=100)
  Generated so far: 10/100
  Generated so far: 20/100
  Generated so far: 30/100
  Generated so far: 40/100
  Generated so far: 50/100
  Generated so far: 60/100
  Generated so far: 70/100
  Generated so far: 80/100
  Generated so far: 90/100
  Generated so far: 100/100
Generating for intent: 주문_결제 (target count=100)
  Generated so far: 10/100
  Generated so far: 20/100
  Generated so far: 30/100
  Generated so far: 40/100
  Generated so far: 50/100
  Generated so far: 60/100
  Generated so far: 70/100
  Generated so far: 80/100
  Generated so far: 90/100
  Generated so far: 100/100
Generating for intent: 배송_문의 (target count=100)
  Generated so far: 10/100
  Generated so far: 20/100
  Generated so far: 30/100
  Generated so far: 40/100
  Generated so far: 50/100
  Generated so far: 60/100
  Generated so far: 70/100
  Generated so far: 80/100
  Generated so far: 90/100
  Generated so far: 100/100
Generating for intent: 상품_정보 (target count=

### 생성된 데이터 확인
생성된 CSV 파일을 pandas DataFrame으로 로드하여 상위 10개 행을 출력하고, 총 데이터 수를 확인합니다. 이를 통해 데이터가 의도한 형식과 내용으로 정상적으로 생성되었는지 검증합니다. 또한, JSONL 파일의 내용도 일부 출력하여 형식을 확인합니다.

In [12]:
# 셀 5: 생성된 CSV/JSONL 파일을 확인하고 간단히 미리보기
import pandas as pd
csv_path = "intent_dataset.csv"
jsonl_path = "intent_dataset.jsonl"


print("CSV 존재 여부:", bool(pd.io.common.file_exists(csv_path)))
if pd.io.common.file_exists(csv_path):
    df = pd.read_csv(csv_path)
    display(df.head(10))
    print("총 행 수:", len(df))

# JSONL도 확인
if pd.io.common.file_exists(jsonl_path):
    with open(jsonl_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    print("JSONL 샘플 (최초 5줄):")
    print("".join(lines[:5]))
    print("총 JSONL 라인 수:", len(lines))

CSV 존재 여부: True


,intent,text,complexity,suggested_model
0,계정_관리,내 계정 비밀번호를 변경하고 싶어요.,simple,small
1,계정_관리,계정을 완전히 삭제하려면 어떻게 해야 하나요?,simple,small
2,계정_관리,2단계 인증을 설정하는 방법을 알려주세요.,simple,small
3,계정_관리,로그인 시 '계정 정지' 메시지가 나오는데 해결 방법이 있나요?,complex,large
4,계정_관리,가족 계정을 추가하려면 어떤 절차가 필요한가요?,complex,large
5,계정_관리,"해외에서 로그인 시도가 감지되었는데, 보안 조치가 가능한가요?",complex,large
6,계정_관리,계정 복구 이메일을 변경하려면 어떻게 해야 하나요?,simple,small
7,계정_관리,연동된 SNS 계정을 해제하는 방법을 모르겠어요.,simple,small
8,계정_관리,계정 활동 내역을 확인하고 의심스러운 로그를 신고하려면?,complex,large
9,계정_관리,결제 수단 정보를 업데이트하려면 어디로 가야 하나요?,simple,small


총 행 수: 454
JSONL 샘플 (최초 5줄):
{"intent": "계정_관리", "text": "내 계정 비밀번호를 변경하고 싶어요.", "complexity": "simple", "suggested_model": "small"}
{"intent": "계정_관리", "text": "계정을 완전히 삭제하려면 어떻게 해야 하나요?", "complexity": "simple", "suggested_model": "small"}
{"intent": "계정_관리", "text": "2단계 인증을 설정하는 방법을 알려주세요.", "complexity": "simple", "suggested_model": "small"}
{"intent": "계정_관리", "text": "로그인 시 '계정 정지' 메시지가 나오는데 해결 방법이 있나요?", "complexity": "complex", "suggested_model": "large"}
{"intent": "계정_관리", "text": "가족 계정을 추가하려면 어떤 절차가 필요한가요?", "complexity": "complex", "suggested_model": "large"}

총 JSONL 라인 수: 454


### 데이터셋 로컬 다운로드 (선택 사항)
Google Colab 환경에서 생성된 데이터셋 파일(intent_dataset.csv, intent_dataset.jsonl)을 로컬 머신으로 다운로드하는 코드입니다. 이 단계는 백업 또는 다른 환경에서의 추가 분석을 위해 선택적으로 실행할 수 있습니다.

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# 셀 6: 생성된 파일을 로컬로 다운로드
from google.colab import files
import os
import shutil

# 로컬 다운로드 (선택 사항)
files.download('intent_dataset.csv')
files.download('intent_dataset.jsonl')

# Google Drive에 저장

# Google Drive가 마운트되어 있어야 합니다. (이전 셀에서 마운트)
drive_dataset_path = "/content/drive/intent_classification_dataset"
os.makedirs(drive_dataset_path, exist_ok=True)

# 파일 복사
csv_path = "intent_dataset.csv"
jsonl_path = "intent_dataset.jsonl"

if os.path.exists(csv_path):
    shutil.copy(csv_path, drive_dataset_path)
    print(f"Copied {csv_path} to {drive_dataset_path}")

if os.path.exists(jsonl_path):
    shutil.copy(jsonl_path, drive_dataset_path)
    print(f"Copied {jsonl_path} to {drive_dataset_path}")

print(f"Dataset files saved to Google Drive: {drive_dataset_path}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

OSError: [Errno 95] Operation not supported: '/content/drive/intent_classification_dataset'

# 02. 생성 데이터셋을 활용한 PEFT 활용 모델 Fine-Tuning
이번 세션에서 사용할 모델은 Smollm2-135m으로, 본래 SequenceClassification을 위한 가중치가 존재하지 않습니다. 이에 이렇게 불러오는 모델의 끝단은 초기화된 가중치를 가진 Score 레이어가 존재하게 되며, 이번 훈련에서 해당 레이어를 포함한 다른 Linear 레이어들을 LoRA를 활용해 훈련합니다.

### 모델 훈련을 위한 라이브러리 임포트
모델 미세조정(Fine-tuning)에 필요한 라이브러리들을 임포트합니다. sklearn은 데이터 분할, torch는 딥러닝 모델의 기반, transformers는 사전 학습된 모델과 토크나이저를 로드하기 위해 사용됩니다. peft 라이브러리는 LoRA와 같은 파라미터 효율적 미세조정 기법을 적용하기 위해 필요합니다. 재현성을 위해 시드(SEED) 값을 고정합니다.

In [23]:
# 셀 2: 기본 임포트 및 설정
import os
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from datasets import Dataset, DatasetDict

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    TrainerCallback,
)

# 재현성
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

In [24]:
import os, random, numpy as np, pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, PeftModel

# reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); os.environ["PYTHONHASHSEED"]=str(SEED)

# 유틸: 모델의 모듈 이름들을 출력/검색해서 LoRA 타겟 후보를 추천
def list_module_names(model, max_items=200):
    names = []
    for n, m in model.named_modules():
        names.append(n)
    # 상위 일부만 출력
    print("=== first 200 module names (truncated) ===")
    for i, nm in enumerate(names[:max_items]):
        print(i, nm)
    return names

def suggest_target_modules_from_names(names):
    # 흔히 사용하는 키워드 기반 추천
    candidates = set()
    keywords = ["q_proj","k_proj","v_proj","o_proj","q","k","v","gate","down_proj","up_proj","dense","proj","wq","wk","wv","wo","attn"]
    for nm in names:
        for kw in keywords:
            if kw in nm:
                candidates.add(nm.split('.')[-1])  # 마지막 파트 기준으로 제안
    # 정렬/중복 제거
    return sorted(list(candidates))[:20]


### 학습 데이터 로드 및 레이블 인코딩
앞서 생성한 intent_dataset.csv 파일을 pandas DataFrame으로 로드합니다. 훈련 태스크의 목표 변수(target variable)인 suggested_model 컬럼의 텍스트 레이블('small', 'large')을 모델이 이해할 수 있도록 숫자(0, 1)로 변환하는 label2id 및 id2label 맵을 정의합니다.



In [15]:
# 필요시 업로드할 수 있게 처리
csv_path = "intent_dataset.csv"
if not os.path.exists(csv_path):
    from google.colab import files
    print("upload intent_dataset.csv (or similarly named file)")
    uploaded = files.upload()
    csv_path = list(uploaded.keys())[0]

df = pd.read_csv(csv_path)
print("loaded", len(df), "rows; sample:")
display(df.head())
# label encoding
label2id = {"small": 0, "large": 1}
id2label = {0: "small", 1: "large"}


loaded 454 rows; sample:


,intent,text,complexity,suggested_model
0,계정_관리,내 계정 비밀번호를 변경하고 싶어요.,simple,small
1,계정_관리,계정을 완전히 삭제하려면 어떻게 해야 하나요?,simple,small
2,계정_관리,2단계 인증을 설정하는 방법을 알려주세요.,simple,small
3,계정_관리,로그인 시 '계정 정지' 메시지가 나오는데 해결 방법이 있나요?,complex,large
4,계정_관리,가족 계정을 추가하려면 어떤 절차가 필요한가요?,complex,large



### 데이터셋 전처리 및 PyTorch DataLoader 생성

모델 학습을 위한 데이터 준비 과정을 수행합니다. 이 단계는 다음의 하위 과정으로 구성됩니다:

PyTorch Dataset 클래스 정의: torch.utils.data.Dataset을 상속받아 커스텀 IntentDataset 클래스를 정의합니다. 이 클래스는 데이터를 토큰화하고 모델 입력 형식에 맞는 텐서(tensor)로 변환하는 역할을 합니다.

데이터 분할: 전체 데이터셋을 sklearn의 train_test_split을 사용하여 학습(train), 검증(validation), 테스트(test) 세트로 8:1:1 비율로 계층적 분할(stratified split)합니다.

데이터셋 및 DataLoader 인스턴스화: 분할된 각 데이터셋에 대해 IntentDataset과 DataLoader를 생성합니다. DataLoader는 배치(batch) 단위로 데이터를 모델에 효율적으로 공급하는 역할을 합니다.

In [16]:
# [문제]: PyTorch의 Dataset 클래스를 상속받아 커스텀 데이터셋을 완성합니다.
class IntentDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        # 1. 레이블 데이터를 텐서로 변환하여 저장합니다.
        self.labels = torch.tensor(dataframe['label_id'].values, dtype=torch.long)
        # [문제 3]: __init__ 메서드에서 데이터프레임의 'text' 컬럼 전체를 한 번에 토크나이징하여
        # self.encodings에 저장합니다. 이는 학습 전, 미리 토크나이징을 진행하여 효율을 크게 높이는 중요한 처리 방식입니다.
        # truncation, padding, max_length, return_tensors='pt' 옵션을 사용해야 합니다.
        # [START CODE]
        self.encodings = tokenizer(
            list(dataframe['text']),
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        # [END CODE]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # [문제 4]: __getitem__ 메서드에서는 미리 토크나이징된 self.encodings에서
        # idx에 해당하는 데이터를 가져와야 합니다.
        # 올바른 텐서 조각(slice)을 가져와 'labels' 키에 레이블을 추가하여 딕셔너리 형태로 반환하세요.
        # [START CODE]
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        # [END CODE]

        return item


### 사전 학습된 모델 및 토크나이저 로드
HuggingFace Hub로부터 HuggingFaceTB/SmolLM2-135M-Instruct 모델을 로드합니다. 이 모델은 시퀀스 분류(Sequence Classification) 태스크를 수행할 수 있도록 AutoModelForSequenceClassification 클래스를 사용하여 초기화하며, num_labels와 레이블 맵(id2label, label2id)을 지정합니다. 해당 모델에 맞는 토크나이저 또한 AutoTokenizer를 통해 로드합니다.

In [17]:

# train/val/test split (80/10/10)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader


SEED = 42

# ---------------------------
# 1. Binary 라벨 매핑
# ---------------------------
df['label_id'] = df['suggested_model'].map({'small': 0, 'large': 1})

# ---------------------------
# [문제 5]: 전체 데이터프레임(df)을 학습+검증(train_val) 데이터와 테스트(test_df) 데이터로 9:1 비율로 분할합니다.
# stratify 옵션을 사용하여 레이블 비율을 유지해야 합니다.
# ---------------------------
# [START CODE]
train_val, test_df = train_test_split(
    df, test_size=0.1, stratify=df['label_id'], random_state=SEED
)
# [END CODE]

# ---------------------------
# [문제 6]: 위에서 나눈 학습+검증(train_val) 데이터를 다시 학습(train_df)과 검증(val_df) 데이터로 분할합니다.
# 학습 데이터의 10%가 검증 데이터가 되도록 test_size를 조절해야 합니다.
# ---------------------------
# [START CODE]
train_df, val_df = train_test_split(
    train_val, test_size=0.1, stratify=train_val['label_id'], random_state=SEED
)

# ---------------------------
# 3. Tokenizer
# ---------------------------
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_length = 128
# [END CODE]


# ---------------------------
# 4. Dataset
# ---------------------------
train_ds = IntentDataset(train_df, tokenizer, max_length=max_length)
val_ds   = IntentDataset(val_df, tokenizer, max_length=max_length)
test_ds  = IntentDataset(test_df, tokenizer, max_length=max_length)

print("train/val/test:", len(train_ds), len(val_ds), len(test_ds))

# ---------------------------
# 5. DataLoader 연결
# ---------------------------
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=16)
test_loader  = DataLoader(test_ds, batch_size=16)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

train/val/test: 367 41 46


### LoRA(Low-Rank Adaptation) 설정 적용
파라미터 효율적 미세조정(PEFT) 기법인 LoRA를 모델에 적용합니다. LoraConfig를 통해 LoRA의 주요 하이퍼파라미터(r, lora_alpha, target_modules 등)를 설정합니다. get_peft_model 함수를 사용하여 기본 모델(base_model)에 LoRA 설정을 적용한 PEFT 모델을 생성합니다. print_trainable_parameters를 통해 전체 파라미터 대비 학습 대상 파라미터의 비율을 확인하여 LoRA가 얼마나 효율적인지 검증합니다.

In [18]:
MODEL_NAME = "HuggingFaceTB/SmolLM2-135M-Instruct"  # 또는 사용하실 정확한 허브 네임 (변경 가능)
# 일부 사용자 업로드/비공식 model은 trust_remote_code=True 가 필요할 수 있음
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
# [문제 7]: HuggingFace의 AutoModelForSequenceClassification을 사용하여 사전 학습된 모델을 로드합니다.
# num_labels, id2label, label2id를 파라미터로 전달하여 분류 태스크에 맞게 모델을 초기화해야 합니다.
# [START CODE]
base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
# [END CODE]
print("Loaded AutoModelForSequenceClassification successfully.")

# [문제 8]: LoRA 설정을 위한 LoraConfig를 정의합니다.
# r=8, lora_alpha=32, 타겟으로는 "all-linear"를 설정하고, 태스크 유형을 시퀀스 분류("SEQ_CLS")로 지정합니다.
# [START CODE]
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules="all-linear",
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"   # 시퀀스 분류
)
# [END CODE]

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # 학습 가능한 파라미터 요약 확인


config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM2-135M-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded AutoModelForSequenceClassification successfully.
trainable params: 2,443,392 || all params: 136,959,552 || trainable%: 1.7840


### 모델 학습 실행
PyTorch를 사용한 표준적인 학습 루프(training loop)를 구현하여 모델을 학습시킵니다.

준비: 모델을 GPU로 이동시키고, AdamW 옵티마이저와 CrossEntropyLoss 손실 함수를 정의합니다.

학습 루프: 지정된 에포크(epoch) 수만큼 반복하며, 각 에포크마다 train_loader로부터 배치 단위로 데이터를 받아 순전파(forward pass), 손실 계산, 역전파(backward pass), 옵티마이저 스텝을 실행합니다.

검증 루프: 각 에포크의 학습이 끝난 후, val_loader를 사용하여 검증 데이터셋에 대한 모델의 성능(손실 및 정확도)을 평가합니다. 이를 통해 과적합(overfitting) 여부를 모니터링합니다.

In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# ---------------------------
# 1. label 처리: small=0, large=1
# ---------------------------
df['label_id'] = df['suggested_model'].map({'small': 0, 'large': 1})
print(df[['suggested_model','label_id']].head(10))


# ---------------------------
# 2. Train/Val/Test split
# ---------------------------
SEED = 42
train_val, test_df = train_test_split(df, test_size=0.1, stratify=df['label_id'], random_state=SEED)
train_df, val_df = train_test_split(train_val, test_size=0.1111111, stratify=train_val['label_id'], random_state=SEED)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
max_length = 128

train_ds = IntentDataset(train_df, tokenizer, max_length=max_length)
val_ds   = IntentDataset(val_df, tokenizer, max_length=max_length)
test_ds  = IntentDataset(test_df, tokenizer, max_length=max_length)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=16)
test_loader  = DataLoader(test_ds, batch_size=16)

# ---------------------------
# 4. Model + optimizer
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
criterion = torch.nn.CrossEntropyLoss()  # Binary classification도 CrossEntropyLoss 사용 가능

# ---------------------------
# 5. Training Loop
# ---------------------------
num_epochs = 5
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    epoch_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # [문제 9]: 모델의 순전파, 손실 계산, 역전파 과정을 완성합니다.
        # 1. 모델에 input_ids와 attention_mask를 전달하여 출력을 얻습니다.
        # 2. CrossEntropyLoss를 사용하여 모델의 출력(logits)과 실제 레이블(labels) 간의 손실을 계산합니다.
        # 3. 계산된 손실에 대해 역전파를 수행합니다.
        # 4. 옵티마이저를 사용하여 모델의 가중치를 업데이트합니다.
        # [START CODE]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        # [END CODE]

        epoch_loss += loss.item() * input_ids.size(0)
        preds = logits.argmax(dim=-1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    print(f"Train loss: {epoch_loss/total:.4f}, accuracy: {correct/total:.4f}")

    # ---------------------------
    # Validation
    # ---------------------------
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)

            val_loss += loss.item() * input_ids.size(0)
            preds = logits.argmax(dim=-1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    print(f"Val   loss: {val_loss/val_total:.4f}, accuracy: {val_correct/val_total:.4f}")
    model.train()


  suggested_model  label_id
0           small         0
1           small         0
2           small         0
3           large         1
4           large         1
5           large         1
6           small         0
7           small         0
8           large         1
9           small         0
Epoch 1/5


100%|██████████| 23/23 [00:13<00:00,  1.70it/s]


Train loss: 0.4365, accuracy: 0.8204
Val   loss: 0.4367, accuracy: 0.8261
Epoch 2/5


100%|██████████| 23/23 [00:11<00:00,  2.05it/s]


Train loss: 0.3334, accuracy: 0.8453
Val   loss: 0.3560, accuracy: 0.8913
Epoch 3/5


...

Train loss: 0.2114, accuracy: 0.9254
Val   loss: 0.2391, accuracy: 0.9130
Epoch 4/5


100%|██████████| 23/23 [00:12<00:00,  1.78it/s]


Train loss: 0.1034, accuracy: 0.9641
Val   loss: 0.2835, accuracy: 0.9130
Epoch 5/5


100%|██████████| 23/23 [00:11<00:00,  2.04it/s]


Train loss: 0.0630, accuracy: 0.9779
Val   loss: 0.2645, accuracy: 0.9130


### 학습된 모델 가중치 저장
학습이 완료된 모델의 가중치를 저장하여 추후 재사용 및 배포가 가능하도록 합니다. 이 셀에서는 Google Drive를 마운트하여 Colab 세션이 종료된 후에도 파일이 유지되도록 합니다.

LoRA 어댑터 저장: save_pretrained 메소드를 사용하여 학습된 LoRA 가중치(어댑터)만 별도로 저장합니다. 이는 용량이 작아 관리가 용이합니다.

모델 병합 및 저장: PeftModel을 사용하여 기본 모델에 학습된 LoRA 어댑터를 병합(merge)합니다. 병합된 전체 모델은 추론(inference) 시 추가적인 처리 없이 바로 사용할 수 있는 상태가 되며, 이 모델을 Google Drive에 저장하여 영속성을 확보합니다.

- 구글 Drive를 사용하지 않는다면, 모델 가중치를 저장해서 로컬등에 받아두어야 합니다.

In [20]:
import os

# adapter만 저장
peft_save_dir = "peft_smollm2_adapters"
model.save_pretrained(peft_save_dir)
print("Saved LoRA adapters to:", peft_save_dir)

# Define the path to save the model in Google Drive
drive_save_path = "/content/drive/MyDrive/smollm2_merged_for_inference"

# Create the directory in Google Drive if it doesn't exist
os.makedirs(drive_save_path, exist_ok=True)

# [문제 10]: 학습된 LoRA 어댑터를 기본 모델과 병합하여 배포용 모델을 생성합니다.
# merge_and_unload() 함수를 사용하세요.
# [START CODE]
# 배포/인퍼런스용으로 base + adapter 합치기
merged = PeftModel.from_pretrained(base_model, peft_save_dir)
merged_model = merged.merge_and_unload()
# [END CODE]

# Save the merged model and tokenizer to Google Drive
merged_model.save_pretrained(drive_save_path)
tokenizer.save_pretrained(drive_save_path)
print(f"Merged model saved to {drive_save_path}")


Saved LoRA adapters to: peft_smollm2_adapters


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Merged model saved to /content/smollm2_merged_for_inference


### 분류 헤드(Classification Head) 가중치 저장
미세조정 과정에서 학습된 부분 중, 최종 분류를 담당하는 score 레이어(분류 헤드)의 가중치만 별도로 저장합니다. 이는 모델의 특정 부분만 분석하거나 다른 모델에 이식할 때 유용할 수 있습니다.

In [ ]:
torch.save(merged_model.score.state_dict(), 'score_only.pt')
merged_model.score

Linear(in_features=576, out_features=2, bias=False)

### 저장된 모델을 이용한 추론(Inference) 테스트
저장된 병합 모델을 다시 불러와 실제 데이터에 대한 예측 성능을 테스트합니다.

모델 로드: Google Drive에 저장된 모델과 토크나이저를 로드합니다. 로딩 실패 시, 기본 모델에 어댑터를 다시 적용하는 대체 로직이 포함되어 있습니다.

예측 함수 정의: 텍스트를 입력받아 토큰화하고, 모델을 통해 예측된 레이블('small' 또는 'large')을 반환하는 predict_suggested_model 함수를 정의합니다.

성능 검증: 사전에 정의된 테스트 문장들에 대해 예측을 수행하고, 예측 결과와 정답을 비교하여 모델이 새로운 데이터에 대해 얼마나 잘 일반화하는지 확인합니다.

In [ ]:
# 셀 8: 모델 추론 예시
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel

# 저장된 모델 경로
drive_save_path = "/content/drive/MyDrive/smollm2_merged_for_inference"

# 저장된 merged model과 tokenizer 로드
try:
    inference_model = AutoModelForSequenceClassification.from_pretrained(drive_save_path)
    inference_tokenizer = AutoTokenizer.from_pretrained(drive_save_path)
    print("Merged model loaded successfully for inference.")
except Exception as e:
    print(f"Failed to load merged model from {drive_save_path}: {e}")
    print("LoRA adapter와 base model을 따로 로드하여 테스트합니다.")
    # 만약 merged model 로딩에 실패하면, base model에 adapter를 연결하여 사용
    base_model_path = "HuggingFaceTB/SmolLM2-135M-Instruct" # base model 경로
    base_model_inf = AutoModelForSequenceClassification.from_pretrained(
        base_model_path,
        num_labels=2,
        id2label=id2label, # 이전 셀에서 정의된 id2label 사용
        label2id=label2id, # 이전 셀에서 정의된 label2id 사용
        trust_remote_code=True
    )
    peft_save_dir = "peft_smollm2_adapters" # adapter 경로 (이전 셀에서 저장한 경로)
    inference_model = PeftModel.from_pretrained(base_model_inf, peft_save_dir)
    inference_model = inference_model.merge_and_unload() # 다시 merge 시도
    inference_tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
    print("Loaded base model and adapter, then merged for inference.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inference_model.to(device)
inference_model.eval() # 평가 모드

# 예측 함수
def predict_suggested_model(text):
    inputs = inference_tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    ).to(device)
    # [문제 11]: 모델 추론 과정을 완성합니다.
    # 1. torch.no_grad() 컨텍스트 내에서 모델의 순전파를 수행하여 출력을 얻습니다.
    # 2. 출력(logits)에서 가장 높은 값을 가진 인덱스를 찾아 예측 결과를 생성합니다.
    # [START CODE]
    with torch.no_grad():
        outputs = inference_model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
    # [END CODE]

    predicted_id = predictions.item()
    predicted_label = inference_model.config.id2label[predicted_id]
    return predicted_label

# 예시 테스트와 정답 비교
# 복잡도 및 suggested_model 정보는 예시 리스트에 직접 명시합니다.
test_sentences_with_info = [
    {"text": "회원가입은 어떻게 하나요?", "complexity": "simple", "correct_model": "small"}, # simple account
    {"text": "주문한 상품의 배송 상태를 추적하고 싶습니다.", "complexity": "simple", "correct_model": "small"}, # simple delivery
    {"text": "결제 시 사용 가능한 할인 쿠폰이 있나요?", "complexity": "simple", "correct_model": "large"}, # simple order (DEFAULT_ROUTER['주문_결제'] == False)
    {"text": "로그인 시도 시 오류 코드가 발생하는데 해결 방법은 무엇인가요?", "complexity": "complex", "correct_model": "large"}, # complex tech support
    {"text": "주문 후 결제 수단을 변경할 수 있나요?", "complexity": "complex", "correct_model": "large"}, # complex order
    {"text": "이 제품의 상세 스펙과 사용 후기를 알고 싶습니다.", "complexity": "complex", "correct_model": "large"} # complex product info
]

print("\n--- Test Predictions ---")
for item in test_sentences_with_info:
    sentence = item['text']
    complexity = item['complexity']
    correct_label = item['correct_model']

    predicted_model = predict_suggested_model(sentence)

    is_correct = "Correct" if predicted_model == correct_label else "Incorrect"

    print(f"'{sentence}'")
    print(f"  -> Complexity: {complexity}, Correct Model: {correct_label}, Predicted: {predicted_model}, Result: {is_correct}")

Failed to load merged model from /content/smollm2_merged_for_inference: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/smollm2_merged_for_inference'. Use `repo_type` argument if needed.
LoRA adapter와 base model을 따로 로드하여 테스트합니다.


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM2-135M-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded base model and adapter, then merged for inference.

--- Test Predictions ---
'회원가입은 어떻게 하나요?'
  -> Complexity: simple, Correct Model: small, Predicted: small, Result: Correct
'주문한 상품의 배송 상태를 추적하고 싶습니다.'
  -> Complexity: simple, Correct Model: small, Predicted: large, Result: Incorrect
'결제 시 사용 가능한 할인 쿠폰이 있나요?'
  -> Complexity: simple, Correct Model: large, Predicted: large, Result: Correct
'로그인 시도 시 오류 코드가 발생하는데 해결 방법은 무엇인가요?'
  -> Complexity: complex, Correct Model: large, Predicted: large, Result: Correct
'주문 후 결제 수단을 변경할 수 있나요?'
  -> Complexity: complex, Correct Model: large, Predicted: large, Result: Correct
'이 제품의 상세 스펙과 사용 후기를 알고 싶습니다.'
  -> Complexity: complex, Correct Model: large, Predicted: large, Result: Correct


# 마치며 (Conclusion)
본 과제를 통해 LLM으로 합성 데이터셋을 구축하고 PEFT(LoRA) 기법으로 소형 모델을 최적화하는 엔드투엔드 파이프라인을 구현했습니다. 이 과정에서 데이터 증강 전략의 유효성과 파라미터 효율적 미세조정의 실용성을 확인하며, 기초적인 MLOps 워크플로우를 경험할 수 있었습니다. 여기서 개발된 모델은 자원 분배가 중요한 실제 서비스에 응용될 수 있으며, 향후 더 복잡한 분류 문제로 확장하거나 다양한 모델을 테스트하는 방향으로 발전시킬 수 있습니다. 이번 실습이 최신 AI 기술에 대한 깊이 있는 이해와 실용적 활용 능력을 다지는 계기가 되었기를 바랍니다.